In [1]:
%load_ext autoreload
%autoreload 2
import os
import os.path as osp
import torch, torchvision
import random
import numpy as np
import PIL.Image as PImage, PIL.ImageDraw as PImageDraw
import torch
import torch.nn
import argparse
import os
import numpy as np
from options import HiDDenConfiguration

import utils
from model.hidden import *
from noise_layers.noiser import Noiser
from PIL import Image
import torchvision.transforms.functional as TF
import torchvision.transforms as transforms

In [2]:
from VAR_model.var_use import ImageEmbedding
embedding_model = ImageEmbedding()

/home/yw699/anaconda3/envs/test/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



[constructor]  ==== flash_if_available=True (0/24), fused_if_available=True (fusing_add_ln=0/24, fusing_mlp=0/24) ==== 
    [VAR config ] embed_dim=1536, num_heads=24, depth=24, mlp_ratio=4.0
    [drop ratios ] drop_rate=0.0, attn_drop_rate=0.0, drop_path_rate=0.1 (tensor([0.0000, 0.0043, 0.0087, 0.0130, 0.0174, 0.0217, 0.0261, 0.0304, 0.0348,
        0.0391, 0.0435, 0.0478, 0.0522, 0.0565, 0.0609, 0.0652, 0.0696, 0.0739,
        0.0783, 0.0826, 0.0870, 0.0913, 0.0957, 0.1000]))

[init_weights] VAR with init_std=0.0147314


/home/yw699/codes/VAR-Watermark/VAR_model/var_use.py:202: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vae.load_state_dict(torch.load(vae_ckpt, map_location='cpu'), strict=

Models loaded and prepared.


In [3]:
device = torch.device('cpu')
options_file = "/home/yw699/codes/VAR-Watermark/runs/watermark 2024.12.17--15-42-23/options-and-config.pickle"
checkpoint_file ="/home/yw699/codes/VAR-Watermark/runs/watermark 2024.12.17--15-42-23/checkpoints/watermark--epoch-300.pyt" 

train_options, hidden_config, noise_config = utils.load_options(options_file)
noiser = Noiser(noise_config,device)

checkpoint = torch.load(checkpoint_file,weights_only=True)
hidden_net = Hidden(hidden_config, device, noiser, None)

utils.model_from_checkpoint(hidden_net, checkpoint)

### Watermark

In [4]:
def add_watermark(m,i):
    m = m.to(device)
    # losses, (encoded_images, noised_images, decoded_messages) = hidden_net.validate_on_batch([image_tensor, message])
    encoded_images = hidden_net.encoder_decoder.encoder(i,m)
    noised_and_cover = hidden_net.encoder_decoder.noiser([encoded_images, i])
    noised_image = noised_and_cover[0]
    return noised_image


def get_watermark(image):
    # losses, (encoded_images, noised_images, decoded_messages) = hidden_net.validate_on_batch([image_tensor, message])
    decoded_message  = hidden_net.encoder_decoder.decoder(image)
    
    return decoded_message 



In [5]:
source_image = "/home/yw699/codes/VAR-Watermark/runs/watermark 2024.12.17--15-42-23/images/epoch-original-300.pt"
#source_image = "/home/yw699/codes/VAR-Watermark/dataset/val/val_class/image0.pt"
batch = 1
image_tensor = torch.load(source_image,weights_only=True)
if image_tensor.dim() == 3:
    image_tensor.unsqueeze_(0)
if image_tensor.dim() == 4 and image_tensor.shape[0] > 10:
    batch = 10
    image_tensor = image_tensor[:batch]
image_tensor = image_tensor.to(device)
print("Original Tensor Shape:", image_tensor.shape)



Original Tensor Shape: torch.Size([10, 32, 16, 16])


### 1.直接加水印

In [6]:
message = torch.Tensor(np.random.choice([0, 1], (image_tensor.shape[0],
                                                    hidden_config.message_length))).to(device)

encoded_images = add_watermark(message,image_tensor)

In [7]:
decoded_message  = get_watermark(encoded_images)

In [8]:
decoded_message  = get_watermark(encoded_images)
decoded_rounded = decoded_message.detach().cpu().numpy().round().clip(0, 1)
message_detached = message.detach().cpu().numpy()
#print('decoded : {}'.format(decoded_rounded))
print('error : {:.3f}'.format(np.mean(np.abs(decoded_rounded - message_detached))))

bitwise_avg_err = np.sum(np.abs(decoded_rounded - message.detach().cpu().numpy()))/(image_tensor.shape[0] * message.shape[1])
print(f"bitwise_avg_err: {bitwise_avg_err}")

error : 0.003
bitwise_avg_err: 0.0033333334140479565


### 2.从图片提取水印

In [9]:
source_image = "/home/yw699/codes/VAR-Watermark/runs/watermark 2024.12.17--15-42-23/images/epoch-original-300.pt"
#source_image = "/home/yw699/codes/VAR-Watermark/dataset/val/val_class/image0.pt"
batch = 1
image_tensor = torch.load(source_image,weights_only=True)
if image_tensor.dim() == 3:
    image_tensor.unsqueeze_(0)
if image_tensor.dim() == 4 and image_tensor.shape[0] > 10:
    batch = 10
    image_tensor = image_tensor[:batch]
image_tensor = image_tensor.to(device)
print("Original Tensor Shape:", image_tensor.shape)



Original Tensor Shape: torch.Size([10, 32, 16, 16])


#### message & the image with watermark 

In [10]:
message = torch.Tensor(np.random.choice([0, 1], (image_tensor.shape[0],
                                                    hidden_config.message_length))).to(device)

encoded_images = add_watermark(message,image_tensor)
torch.save(encoded_images.cpu(), "encoded_images.pt") 
input_path_original = "encoded_images.pt"
output ='watermark_image.png'
image = embedding_model.embed_to_image(input_path_original,output)


Embedding Tensor Shape: torch.Size([10, 32, 16, 16])
Image saved to watermark_image.png


#### Read the decoded message from this image

In [11]:
watermark_image_tensor = embedding_model.image_to_embed(output,"f.pt",batch).to(device)
print("watermark_image_tensor:", watermark_image_tensor.shape)

decoded_message  = get_watermark(watermark_image_tensor)

(2560, 256)
Image Tensor Shape: torch.Size([10, 3, 256, 256])
Embedding saved to f.pt
watermark_image_tensor: torch.Size([10, 32, 16, 16])


In [12]:
decoded_message  = get_watermark(encoded_images)
decoded_rounded = decoded_message.detach().cpu().numpy().round().clip(0, 1)
message_detached = message.detach().cpu().numpy()

print('error : {:.3f}'.format(np.mean(np.abs(decoded_rounded - message_detached))))

bitwise_avg_err = np.sum(np.abs(decoded_rounded - message.detach().cpu().numpy()))/(image_tensor.shape[0] * message.shape[1])
print(f"bitwise_avg_err: {bitwise_avg_err}")

error : 0.017
bitwise_avg_err: 0.01666666753590107
